In [1]:
import pandas as pd
import pandas_datareader.data as web

from datetime import date, timedelta, datetime

from bs4 import BeautifulSoup
import requests

df_main = pd.read_csv('https://squeezemetrics.com/monitor/static/DIX.csv',
                      names=['Date','SPX','DIX','GEX'], index_col='Date', parse_dates=['Date'], header=0)

start = datetime(2011,1,1)
end = datetime.today()

df_spy = web.DataReader(
    'SPY','yahoo', start, end).rename(columns={'Volume' : 'SPY Volume'})[['SPY Volume']]

df_uso = web.DataReader(
    'USO','yahoo', start, end).rename(columns={'Adj Close' : 'USO Price'})[['USO Price']]

df_gld = web.DataReader(
    'GLD','yahoo', start, end).rename(columns={'Adj Close' : 'GLD Price'})[['GLD Price']]

df_vix = pd.read_csv('http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv',
                     skiprows=1, index_col='Date',
                     parse_dates=['Date']).rename(columns={'VIX Close' : 'VIX'})[['VIX']]

/home/cjporteo/.local/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
# updates CBOE info
# requires CBOE-scrape-build.ipynb to be executed at least once before

df_cboe_base = pd.read_csv('CBOE-data.csv', index_col='Date', parse_dates=['Date'])
cboe_data_dict = {}

n = len(df_cboe_base)
last_known_date = df_cboe_base.index[n-1]

curr_date = last_known_date
delta = timedelta(days=1)

while curr_date <= end:
    
    y, m, d = curr_date.year, curr_date.month, curr_date.day
    print(f'Scraping date: {y}-{m:02}-{d:02}')
    
    url = f'https://markets.cboe.com/us/options/market_statistics/daily/?mkt=cone&dt={y}-{m}-{d}'
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    
    date_id = 'stats-date-header'
    date_string = soup.find(id=date_id).get_text()
    date_string = ' '.join(date_string.split(' ')[-3:])
    
    page_date = datetime.strptime(date_string, "%B %d, %Y")
    
    table_class = 'bats-table bats-table--left'
    tables = soup.findAll('table', {'class' : table_class})
    
    # to find the table indices
    # Summary: 0, VIX: 5, SPX + SPXW: 6
    summary_index, vix_index, spx_index = -1, -1, -1
    for i,t in enumerate(tables):
        s = str(t)
        if 'RATIOS' in s:
            summary_index = i
        elif 'CBOE VOLATILITY INDEX (VIX)' in s:
            vix_index = i
        elif 'SPX + SPXW' in s:
            spx_index = i
        if min([summary_index, vix_index, spx_index]) >= 0:
            break
    
    table_summary = tables[summary_index]
    table_vix = tables[vix_index]
    table_spx = tables[spx_index]
    
    df_summary = pd.read_html(str(table_summary))[0]
    df_vix_info = pd.read_html(str(table_vix), skiprows=1)[0]
    df_spx_info = pd.read_html(str(table_spx), skiprows=1)[0]
        
    vix_pc_ratio = df_summary.iloc[4,1]
    spx_pc_ratio = df_summary.iloc[5,1]
    
    vix_volume = df_vix_info.iloc[0,3]
    spx_volume = df_spx_info.iloc[0,3]
    
    cboe_data_dict[page_date] = [
        vix_pc_ratio,
        vix_volume,
        spx_pc_ratio,
        spx_volume
    ]
    
    curr_date += delta

df_cboe_new = pd.DataFrame.from_dict(
    cboe_data_dict,
    orient='index',
    columns=['VIX P/C Ratio', 'VIX Options Volume', 'SPX P/C Ratio', 'SPX Options Volume']
)

df_cboe_new.index.rename('Date', inplace=True)

df_cboe_full = pd.concat([df_cboe_base, df_cboe_new]).reset_index()
df_cboe_full = df_cboe_full.drop_duplicates(subset='Date').set_index('Date').sort_index()
df_cboe_full.to_csv('CBOE-data.csv')

df_cboe = pd.read_csv('CBOE-data.csv', index_col='Date', parse_dates=['Date'])


Scraping date: 2020-06-11
Scraping date: 2020-06-12
Scraping date: 2020-06-13
Scraping date: 2020-06-14


In [3]:
# get treasury info
url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldAll'
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')

table = soup.find('table', {'class' : 't-chart'})
df_treasury = pd.read_html(str(table), index_col='Date', parse_dates=['Date'])[0]

treasury_rename = {col : f'TY {col}' for col in df_treasury.columns}
df_treasury.rename(columns = treasury_rename, inplace=True)

df_treasury = df_treasury[df_treasury.index >= start].dropna(axis=1)

In [4]:
df_main

,SPX,DIX,GEX
Date,,,
2011-05-02,1361.219971,0.378842,1.897313e+09
2011-05-03,1356.619995,0.383411,1.859731e+09
2011-05-04,1347.319946,0.392122,1.717764e+09
2011-05-05,1335.099976,0.405457,1.361864e+09
2011-05-06,1340.199951,0.418649,1.490329e+09
...,...,...,...
2020-06-08,3232.390000,0.461516,5.719837e+09
2020-06-09,3207.180000,0.473334,8.317776e+09
2020-06-10,3190.140000,0.466881,9.662665e+09


In [5]:
df_spy

,SPY Volume
Date,
2011-01-03,138725200.0
2011-01-04,137409700.0
2011-01-05,133975300.0
2011-01-06,122519000.0
2011-01-07,156034600.0
...,...
2020-06-08,73641200.0
2020-06-09,77479200.0
2020-06-10,95000800.0


In [6]:
df_uso

,USO Price
Date,
2011-01-03,312.399994
2011-01-04,304.640015
2011-01-05,308.160004
2011-01-06,301.440002
2011-01-07,301.359985
...,...
2020-06-08,28.370001
2020-06-09,28.420000
2020-06-10,28.430000


In [7]:
df_gld

,GLD Price
Date,
2011-01-03,138.000000
2011-01-04,134.750000
2011-01-05,134.369995
2011-01-06,133.830002
2011-01-07,133.580002
...,...
2020-06-08,159.720001
2020-06-09,161.259995
2020-06-10,163.570007


In [8]:
df_treasury

,TY 1 mo,TY 3 mo,TY 6 mo,TY 1 yr,TY 2 yr,TY 3 yr,TY 5 yr,TY 7 yr,TY 10 yr,TY 20 yr,TY 30 yr
Date,,,,,,,,,,,
2011-01-03,0.11,0.15,0.19,0.29,0.61,1.03,2.02,2.74,3.36,4.18,4.39
2011-01-04,0.12,0.14,0.19,0.28,0.63,1.04,2.01,2.72,3.36,4.21,4.44
2011-01-05,0.13,0.14,0.19,0.31,0.71,1.16,2.14,2.86,3.50,4.34,4.55
2011-01-06,0.13,0.15,0.18,0.30,0.68,1.11,2.09,2.80,3.44,4.31,4.53
2011-01-07,0.13,0.14,0.18,0.29,0.60,1.02,1.96,2.69,3.34,4.25,4.48
...,...,...,...,...,...,...,...,...,...,...,...
2020-06-08,0.15,0.17,0.19,0.19,0.22,0.29,0.45,0.69,0.88,1.43,1.65
2020-06-09,0.14,0.19,0.19,0.19,0.20,0.25,0.40,0.65,0.84,1.37,1.59
2020-06-10,0.13,0.17,0.19,0.18,0.17,0.22,0.33,0.56,0.75,1.31,1.53


In [9]:
df_vix

,VIX
Date,
2004-01-02,18.22
2004-01-05,17.49
2004-01-06,16.73
2004-01-07,15.50
2004-01-08,15.61
...,...
2020-06-08,25.81
2020-06-09,27.57
2020-06-10,27.57


In [10]:
df_cboe

,VIX P/C Ratio,VIX Options Volume,SPX P/C Ratio,SPX Options Volume
Date,,,,
2010-07-07,0.79,184762,0.91,634363
2010-07-08,1.25,141024,1.12,531457
2010-07-09,0.56,196082,1.20,691937
2010-07-12,2.01,241936,1.49,704517
2010-07-13,0.97,159440,1.25,1172241
...,...,...,...,...
2020-06-08,1.56,320229,1.07,1249640
2020-06-09,1.25,372416,1.97,909186
2020-06-10,1.11,267652,1.38,1112532


In [11]:
dfs_to_join = [
    df_spy,
    df_uso,
    df_gld,
    df_treasury,
    df_vix,
    df_cboe
]

df_final = df_main.join(dfs_to_join, how='left').sort_index()
df_final

,SPX,DIX,GEX,SPY Volume,USO Price,GLD Price,TY 1 mo,TY 3 mo,TY 6 mo,TY 1 yr,...,TY 5 yr,TY 7 yr,TY 10 yr,TY 20 yr,TY 30 yr,VIX,VIX P/C Ratio,VIX Options Volume,SPX P/C Ratio,SPX Options Volume
Date,,,,,,,,,,,,,,,,,,,,,
2011-05-02,1361.219971,0.378842,1.897313e+09,126278700.0,359.440002,150.410004,0.02,0.05,0.10,0.22,...,1.96,2.66,3.31,4.14,4.38,15.99,0.51,182366.0,1.38,567584.0
2011-05-03,1356.619995,0.383411,1.859731e+09,138375000.0,352.640015,149.880005,0.02,0.03,0.09,0.20,...,1.96,2.64,3.28,4.11,4.36,16.70,0.39,318780.0,1.84,540934.0
2011-05-04,1347.319946,0.392122,1.717764e+09,182678500.0,346.079987,147.729996,0.02,0.03,0.07,0.19,...,1.95,2.61,3.25,4.08,4.33,17.08,0.87,369293.0,2.08,432621.0
2011-05-05,1335.099976,0.405457,1.361864e+09,226900000.0,314.559998,143.470001,0.01,0.02,0.07,0.20,...,1.88,2.54,3.18,4.00,4.26,18.20,1.36,446203.0,1.94,601038.0
2011-05-06,1340.199951,0.418649,1.490329e+09,222787200.0,310.959991,145.300003,0.02,0.02,0.07,0.18,...,1.87,2.54,3.19,4.03,4.29,18.40,0.41,382407.0,1.85,601669.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-08,3232.390000,0.461516,5.719837e+09,73641200.0,28.370001,159.720001,0.15,0.17,0.19,0.19,...,0.45,0.69,0.88,1.43,1.65,25.81,1.56,320229.0,1.07,1249640.0
2020-06-09,3207.180000,0.473334,8.317776e+09,77479200.0,28.420000,161.259995,0.14,0.19,0.19,0.19,...,0.40,0.65,0.84,1.37,1.59,27.57,1.25,372416.0,1.97,909186.0
2020-06-10,3190.140000,0.466881,9.662665e+09,95000800.0,28.430000,163.570007,0.13,0.17,0.19,0.18,...,0.33,0.56,0.75,1.31,1.53,27.57,1.11,267652.0,1.38,1112532.0


In [12]:
df_final.to_csv('dataset.csv')

In [13]:
df_final.describe()

,SPX,DIX,GEX,SPY Volume,USO Price,GLD Price,TY 1 mo,TY 3 mo,TY 6 mo,TY 1 yr,...,TY 5 yr,TY 7 yr,TY 10 yr,TY 20 yr,TY 30 yr,VIX,VIX P/C Ratio,VIX Options Volume,SPX P/C Ratio,SPX Options Volume
count,2294.000000,2294.000000,2.294000e+03,2.294000e+03,2294.000000,2294.000000,2277.000000,2277.000000,2277.000000,2277.000000,...,2277.000000,2277.000000,2277.000000,2277.000000,2277.000000,2294.000000,2294.000000,2.294000e+03,2294.000000,2.294000e+03
mean,2126.495632,0.420558,2.197821e+09,1.169531e+08,171.168452,131.455820,0.609679,0.646105,0.722336,0.804936,...,1.560110,1.901159,2.211805,2.677699,2.947400,16.982533,0.518326,5.731630e+05,1.738487,1.060877e+06
std,564.792628,0.028476,1.644147e+09,6.835739e+07,94.332131,18.817965,0.808959,0.825054,0.837684,0.839311,...,0.619836,0.551050,0.513095,0.531557,0.529357,7.414574,0.307141,3.538584e+05,0.369990,4.103592e+05
min,1099.229980,0.330555,-2.958423e+09,2.027000e+07,17.040001,100.500000,0.000000,0.000000,0.020000,0.080000,...,0.290000,0.480000,0.540000,0.870000,0.990000,9.140000,0.070000,4.135500e+04,0.810000,2.234330e+05
25%,1679.042511,0.401213,1.173986e+09,7.187170e+07,90.639999,117.985003,0.030000,0.040000,0.080000,0.150000,...,1.130000,1.490000,1.860000,2.380000,2.710000,12.850000,0.320000,3.498890e+05,1.480000,7.627438e+05
50%,2080.380005,0.420087,2.128995e+09,9.910270e+07,117.000000,124.505001,0.110000,0.120000,0.180000,0.320000,...,1.570000,1.960000,2.220000,2.690000,2.980000,14.850000,0.440000,4.890555e+05,1.710000,9.957620e+05
75%,2640.840000,0.438555,3.139562e+09,1.424152e+08,274.079987,146.587498,1.090000,1.260000,1.370000,1.460000,...,1.840000,2.240000,2.600000,2.970000,3.170000,18.337500,0.620000,6.851772e+05,1.970000,1.281210e+06
max,3386.120000,0.522603,1.156640e+10,7.178287e+08,359.440002,184.589996,2.510000,2.490000,2.580000,2.740000,...,3.090000,3.180000,3.310000,4.140000,4.400000,82.690000,3.090000,4.336057e+06,3.770000,3.593415e+06
